<a href="https://colab.research.google.com/github/StavroK/ServiceNow/blob/main/ServiceNow_Incident_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
import keras 
import numpy as np
from keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Flatten
from keras.models import Model
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
plt.style.use('ggplot')
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [72]:
data = pd.read_csv('incident en.csv')

In [73]:
data.head()

,description,number,parent_incident,caller_id,caller_id.location.parent,state,assignment_group,assigned_to,short_description,opened_at,priority,sys_updated_on,business_service,category,closed_at,closed_by,sys_created_on,sys_created_by,opened_by,u_technology
0,Buenos días Estimados:\r\n\r\nM...,INC0026213,NaN,Daniel Ventura Gonzalez,Mexico,Closed,LatAm Applications SD,Oscar Medellin,Creacion de grupo de vendedores,10/10/2018 11:10,Very Low,18/10/2018 08:17,SAP ERP,Inquiry / Help,18/10/2018 08:17,Oscar Medellin,10/10/2018 11:11,oscar.medellin@mexichem.com,Oscar Medellin,SAP LatAm SD
1,CONTRATO DE COMODATO E INVENTÁRI...,INC0430192,NaN,Donalson Harielton Mendes,Brazil,In Progress,Brazil Workplace Inventory,Maiara Paula de Jesus,CONTRATO DE COMODATO E INVENTÁRIO - NOVO COLAB...,07/10/2021 09:59,Low,08/10/2021 15:33,Desktop Non-Managed,Hardware,NaN,NaN,07/10/2021 09:59,donalson.mendes@wavin.com,Donalson Harielton Mendes,NaN
2,Inventário\n\n Inventári...,INC0404409,NaN,Marcelo Dalmolini Nunes,Brazil,Closed,Brazil Workplace Inventory,Maiara Paula de Jesus,Inventário,04/08/2021 14:21,Low,12/08/2021 15:17,Desktop Managed Services,Hardware,12/08/2021 15:17,Maiara Paula de Jesus,04/08/2021 14:21,Marcelo.Nunes@mexichem.com,Marcelo Dalmolini Nunes,Desktop Hardware
3,Por favor necesito de su apoyo u...,INC0031543,NaN,William Lamount Santiago,Venezuela Bolivarian Republic of,Closed,Mexico Service Desk,Misael Arturo Tovar Roman,SAP - Restablecimiento de contraseña.,01/11/2018 13:47,Very Low,08/11/2018 21:17,SAP ERP,request,08/11/2018 21:17,Misael Arturo Tovar Roman,01/11/2018 13:47,william.lamount@mexichem.com,William Lamount Santiago,NaN
4,CONTRATO DE COMODATO E INVENTÁRIO ...,INC0411138,NaN,Marcelo Dalmolini Nunes,Brazil,Closed,Brazil Workplace Inventory,Maiara Paula de Jesus,Gestão de Ativos - Adição de Equipamento,23/08/2021 08:26,Low,02/09/2021 15:17,Desktop Non-Managed,Hardware,02/09/2021 15:17,Maiara Paula de Jesus,23/08/2021 08:26,Marcelo.Nunes@mexichem.com,Marcelo Dalmolini Nunes,NaN


In [74]:
data.category.value_counts()

Software          41730
Inquiry / Help    24298
Access Issue      17699
Hardware           7815
Network            5300
Database            996
ITSM                429
Security            354
request             246
none                 25
Name: category, dtype: int64

In [75]:
data['target'] = data.category.astype('category').cat.codes

In [76]:
data['description'] = data['description'].astype(str)

In [77]:
data.head()

,description,number,parent_incident,caller_id,caller_id.location.parent,state,assignment_group,assigned_to,short_description,opened_at,priority,sys_updated_on,business_service,category,closed_at,closed_by,sys_created_on,sys_created_by,opened_by,u_technology,target
0,Buenos días Estimados:\r\n\r\nM...,INC0026213,NaN,Daniel Ventura Gonzalez,Mexico,Closed,LatAm Applications SD,Oscar Medellin,Creacion de grupo de vendedores,10/10/2018 11:10,Very Low,18/10/2018 08:17,SAP ERP,Inquiry / Help,18/10/2018 08:17,Oscar Medellin,10/10/2018 11:11,oscar.medellin@mexichem.com,Oscar Medellin,SAP LatAm SD,4
1,CONTRATO DE COMODATO E INVENTÁRI...,INC0430192,NaN,Donalson Harielton Mendes,Brazil,In Progress,Brazil Workplace Inventory,Maiara Paula de Jesus,CONTRATO DE COMODATO E INVENTÁRIO - NOVO COLAB...,07/10/2021 09:59,Low,08/10/2021 15:33,Desktop Non-Managed,Hardware,NaN,NaN,07/10/2021 09:59,donalson.mendes@wavin.com,Donalson Harielton Mendes,NaN,2
2,Inventário\n\n Inventári...,INC0404409,NaN,Marcelo Dalmolini Nunes,Brazil,Closed,Brazil Workplace Inventory,Maiara Paula de Jesus,Inventário,04/08/2021 14:21,Low,12/08/2021 15:17,Desktop Managed Services,Hardware,12/08/2021 15:17,Maiara Paula de Jesus,04/08/2021 14:21,Marcelo.Nunes@mexichem.com,Marcelo Dalmolini Nunes,Desktop Hardware,2
3,Por favor necesito de su apoyo u...,INC0031543,NaN,William Lamount Santiago,Venezuela Bolivarian Republic of,Closed,Mexico Service Desk,Misael Arturo Tovar Roman,SAP - Restablecimiento de contraseña.,01/11/2018 13:47,Very Low,08/11/2018 21:17,SAP ERP,request,08/11/2018 21:17,Misael Arturo Tovar Roman,01/11/2018 13:47,william.lamount@mexichem.com,William Lamount Santiago,NaN,9
4,CONTRATO DE COMODATO E INVENTÁRIO ...,INC0411138,NaN,Marcelo Dalmolini Nunes,Brazil,Closed,Brazil Workplace Inventory,Maiara Paula de Jesus,Gestão de Ativos - Adição de Equipamento,23/08/2021 08:26,Low,02/09/2021 15:17,Desktop Non-Managed,Hardware,02/09/2021 15:17,Maiara Paula de Jesus,23/08/2021 08:26,Marcelo.Nunes@mexichem.com,Marcelo Dalmolini Nunes,NaN,2


In [78]:
data['num_words'] = data.description.apply(lambda x: " ".join(x.lower() for x in str(x).split()))

In [79]:
bins=[0,50,75, np.inf]
data['bins']=pd.cut(data.num_words, bins=[0,100,300,500,800, np.inf], labels=['0-100', '100-300', '300-500','500-800' ,'>800'])

TypeError: ignored

In [ ]:
word_distribution = data.groupby('bins').size().reset_index().rename(columns={0:'counts'})

In [ ]:
word_distribution.head()

In [ ]:
sns.barplot(x='bins', y='counts', data=word_distribution).set_title("Word distribution per bin")

In [ ]:
data.head()

In [ ]:
num_class = len(np.unique(data.Category.values))
y = data['target'].values

Topic Modelling

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np
import pandas as pd
import re
import warnings

#Visualisation
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from IPython.display import display
#from mpl_toolkits.basemap import Basemap
from wordcloud import WordCloud, STOPWORDS

#nltk
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize

In [ ]:
data['Description'] = data['Description'].astype(str)

In [ ]:
type(data['Description'])

In [ ]:
data_description = data.Description.values

In [ ]:
type(data_description)

In [ ]:
#cv = # Initialize a CountVectorizer object: count_vectorizer
count_vec = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 1), max_df=1, min_df=1, max_features=None)

# convert text data into term-frequency matrix
data_description = count_vec.fit_transform(data_description)




tfidf_transformer = TfidfTransformer()

# convert term-frequency matrix into tf-idf
tfidf_matrix = tfidf_transformer.fit_transform(data_description)

# create dictionary to find a tfidf word each word
word2tfidf = dict(zip(count_vec.get_feature_names(), tfidf_transformer.idf_))

for word, score in word2tfidf.items():
    print(word, score)

In [ ]:
term_frequency = count_vec.vocabulary_
type(term_frequency)

In [ ]:
sorted_by_idf = np.argsort(tfidf_transformer.idf_)
feature_names = np.array(count_vec.get_feature_names())
print("Features with lowest idf:\n{}".format(
       feature_names[sorted_by_idf[:10]]))
print("\nFeatures with highest idf:\n{}".format(
       feature_names[sorted_by_idf[-10:]]))

In [ ]:
count = nltk.FreqDist(data.Description.values)
count.plot(30)

In [ ]:
final_doc = data['Description']
all_documents = tuple(final_doc)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_documents)
print (tfidf_matrix.shape)
print(tfidf_matrix)

In [ ]:
reindexed_data= data["Description"]

data_text = data[['Description']]
data_text['index'] = data_text.index
documents=data_text


print(len(documents))
print(documents[:5])

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk


def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
from nltk.stem.porter import *
stemmer = PorterStemmer()
stemmer.stem('identified')


doc_sample = documents[documents['index'] == 5].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))


processed_docs = documents['Description'].map(preprocess)
processed_docs[:5]

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[123]

In [ ]:
#Preview Bag Of Words for our sample preprocessed document.
bow_doc_123 = bow_corpus[123]
for i in range(len(bow_doc_123)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_123[i][0], 
                                               dictionary[bow_doc_123[i][0]], 
bow_doc_123[i][1]))

In [ ]:
#TF-IDF
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary, passes=2, workers=2)

In [ ]:
# Print the results
import pickle

# Here we can assign topic names to each topic list, you can specify the number of words you want to have shown (e.g num_words = 5)
result = lda_model.print_topics(num_topics=5, num_words=5)
print(result)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

In [ ]:
# put these in a file and name the topics
with open("lda_topics_result.txt", "w") as f:
    for s in result:
        f.write(str(s) +"\n")

In [ ]:
# Visualize
import pyLDAvis.gensim

lda_display = pyLDAvis.gensim.prepare(lda_model, bow_corpus,dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

In [ ]:
from collections import OrderedDict

data_lda = {i: OrderedDict(lda_model.show_topic(i,25)) for i in range(5)}

In [ ]:
import pandas as pd

df_lda = pd.DataFrame(data_lda)
print(df_lda.shape)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

In [ ]:
df_lda

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

g=sns.clustermap(df_lda.corr(), center=0, cmap="RdBu", metric='cosine', linewidths=.75, figsize=(12, 12))
plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.show()

In [ ]:
MAX_LENGTH = 500
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data.Description.values)
post_seq = tokenizer.texts_to_sequences(data.Description.values)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(post_seq_padded, y, test_size=0.05,random_state=42)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

Deep Learning Simple Model

In [ ]:
inputs = Input(shape=(MAX_LENGTH, ))
embedding_layer = Embedding(vocab_size,
                            128,
                            input_length=MAX_LENGTH)(inputs)
x = Flatten()(embedding_layer)
x = Dense(32, activation='relu')(x)

predictions = Dense(num_class, activation='softmax')(x)
model = Model(inputs=[inputs], outputs=predictions)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()
filepath="weights-simple.hdf5"
checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history = model.fit([X_train], batch_size=64, y=to_categorical(y_train), verbose=1, validation_split=0.25, 
          shuffle=True, epochs=10, callbacks=[checkpointer])

In [ ]:
df = pd.DataFrame({'epochs':history.epoch, 'accuracy': history.history['acc'], 'validation_accuracy': history.history['val_acc']})
g = sns.pointplot(x="epochs", y="accuracy", data=df, fit_reg=False)
g = sns.pointplot(x="epochs", y="validation_accuracy", data=df, fit_reg=False, color='green')

In [ ]:
predicted = model.predict(X_test)
predicted = np.argmax(predicted, axis=1)
accuracy_score(y_test, predicted)

In [ ]:
cnf_matrix = confusion_matrix(y_test, predicted)
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=text_labels, title="Confusion matrix")
plt.show()